In [2]:
!pip install openmeteo-requests
!pip install requests-cache retry-requests numpy pandas

In [4]:
import openmeteo_requests

import pandas as pd
import requests_cache
from retry_requests import retry

# Setup the Open-Meteo API client with cache and retry on error
cache_session = requests_cache.CachedSession('.cache', expire_after = 3600)
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
openmeteo = openmeteo_requests.Client(session = retry_session)

# Make sure all required weather variables are listed here
# The order of variables in hourly or daily is important to assign them correctly below
url = "https://api.open-meteo.com/v1/forecast"
params = {
	"latitude": -6.2186,
	"longitude": 106.803914,
	"hourly": ["temperature_2m", "wind_speed_10m", "wind_direction_10m"],
	"timezone": "Asia/Bangkok",
	"start_date": "2025-06-11",
	"end_date": "2025-06-11"
}
responses = openmeteo.weather_api(url, params=params)

# Process first location. Add a for-loop for multiple locations or weather models
response = responses[0]
print(f"Coordinates {response.Latitude()}°N {response.Longitude()}°E")
print(f"Elevation {response.Elevation()} m asl")
print(f"Timezone {response.Timezone()}{response.TimezoneAbbreviation()}")
print(f"Timezone difference to GMT+0 {response.UtcOffsetSeconds()} s")

# Process hourly data. The order of variables needs to be the same as requested.
hourly = response.Hourly()
hourly_temperature_2m = hourly.Variables(0).ValuesAsNumpy()
hourly_wind_speed_10m = hourly.Variables(1).ValuesAsNumpy()
hourly_wind_direction_10m = hourly.Variables(2).ValuesAsNumpy()

hourly_data = {"date": pd.date_range(
	start = pd.to_datetime(hourly.Time(), unit = "s", utc = True),
	end = pd.to_datetime(hourly.TimeEnd(), unit = "s", utc = True),
	freq = pd.Timedelta(seconds = hourly.Interval()),
	inclusive = "left"
)}

hourly_data["temperature_2m"] = hourly_temperature_2m
hourly_data["wind_speed_10m"] = hourly_wind_speed_10m
hourly_data["wind_direction_10m"] = hourly_wind_direction_10m

hourly_dataframe = pd.DataFrame(data = hourly_data)
print(hourly_dataframe)



Coordinates -6.25°N 106.75°E
Elevation 44.0 m asl
Timezone b'Asia/Bangkok'b'GMT+7'
Timezone difference to GMT+0 25200 s
                        date  temperature_2m  wind_speed_10m  \
0  2025-06-10 17:00:00+00:00       25.934999        5.860375   
1  2025-06-10 18:00:00+00:00       25.584999        4.610250   
2  2025-06-10 19:00:00+00:00       25.385000        4.582052   
3  2025-06-10 20:00:00+00:00       25.184999        4.104631   
4  2025-06-10 21:00:00+00:00       25.135000        3.563818   
5  2025-06-10 22:00:00+00:00       25.084999        3.096837   
6  2025-06-10 23:00:00+00:00       25.035000        2.305125   
7  2025-06-11 00:00:00+00:00       25.385000        2.595997   
8  2025-06-11 01:00:00+00:00       26.785000        3.893995   
9  2025-06-11 02:00:00+00:00       28.434999        6.519877   
10 2025-06-11 03:00:00+00:00       29.635000        9.007196   
11 2025-06-11 04:00:00+00:00       30.684999        9.360000   
12 2025-06-11 05:00:00+00:00       31.434999    

In [5]:
# Convert UTC to GMT+7 (Asia/Jakarta is GMT+7, no DST)
hourly_dataframe["date"] = hourly_dataframe["date"].dt.tz_convert("Asia/Jakarta")

# Filter only 8 PM, 9 PM, and 10 PM
hourly_dataframe_filtered = hourly_dataframe[
    hourly_dataframe["date"].dt.hour.isin([20, 21, 22])
]

# Show the filtered result
print(hourly_dataframe_filtered)

                        date  temperature_2m  wind_speed_10m  \
20 2025-06-11 20:00:00+07:00       27.734999        5.506941   
21 2025-06-11 21:00:00+07:00       27.184999        3.415260   
22 2025-06-11 22:00:00+07:00       26.934999        1.938659   

    wind_direction_10m  
20          101.309898  
21          108.435043  
22          111.801476  
